# Labelling states directly in qubit system

In [3]:
import time
import numpy as np
import cmath
import pysqkit
from typing import Callable
import qutip as qtp
from typing import List

import pysqkit.util.transformations as trf
from pysqkit.util.linalg import hilbert_schmidt, tensor_prod

from IPython.display import display, Latex

In this tutorial we show how to perform tomography on a QubitSystem object. For concreteness we consider a Transmon-Fluxonium system with capacitive coupling. Parameters are taken in order to implement a version of the gate presented in Ficheux et al, Phys. Rev. X 11, 021026 (2021) using the interaction between  $| 1_t 3_f \rangle$ and $| 0_t 4_f \rangle$.

We introduce the Qubit objects

In [4]:
levels_f = 5
levels_t = 3
d_comp = 4
d_leak = levels_t*levels_f - d_comp
flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=.973, 
    induct_energy=.457, 
    joseph_energy=8.0
)

trans = pysqkit.qubits.SimpleTransmon(
    label='T', 
    max_freq=4.5, 
    anharm=-0.3, 
    dim_hilbert=levels_t
)

flx.diagonalize_basis(levels_f)
energies_f, eig_states_f = flx.eig_states(levels_f)
energies_t, eig_states_t = trans.eig_states(levels_t)  

We add a microwave drive on the fluxonium

In [5]:
#Drive
flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cz_drive_f',
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

We capacitively couple the transmon and the fluxonium

In [6]:
jc = 0.07
coupled_sys = trans.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
energies_sys, eigstates_sys = coupled_sys.eig_states() 

coupled_sys_bare = trans.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=0)
energies_bare, eigstates_bare = coupled_sys_bare.eig_states()
    
energies_in_lexico = np.zeros(levels_f*levels_t, dtype=float)
count = 0
for k in range(0, levels_t):
    for m in range(0, levels_f):
        energies_in_lexico[count] = energies_t[k] + energies_f[m]
        count += 1
ascending_to_lexico = np.argsort(energies_in_lexico)
label_converter = ascending_to_lexico

We conveniently label the states

We now fix the drive parameters. These are taken to satisfy the conditions in order for the gate to work.

In [9]:
def label_to_states(
    lev_labels: List[int],
    dims: List[int],
    eig_states: np.ndarray,
    label_converter: np.ndarray 
) -> float:
    
    n_sys = len(lev_labels)
    
    if n_sys != len(dims):
        raise ValueError("The number of levels must be equal to the number of systems")
    
    if np.any(np.array(lev_labels) >= np.array(dims)) == True:
        raise ValueError("Level labels must be smaller than the corresponding dimension")
    label = 0
    factor = 1
    for k in reversed(range(0, n_sys)):
        label += factor*lev_labels[k]
        factor = factor*dims[k]
    index = np.where(label_converter == label)[0][0] #index such that converter[index] = label
    return eig_states[index]

eig_states_by_label_prova = []
energies_by_label_prova = []
for i_a in range(levels_t):
    energies_by_label_prova.append([])
    eig_states_by_label_prova.append([])
    for i_b in range(levels_f):
        eig_states_by_label_prova[i_a].append(qtp.Qobj(inpt=label_to_states([i_a, i_b], [levels_t, levels_f], eigstates_sys, label_converter), 
                                                 dims=[[levels_t, levels_f], [1, 1]], shape=[levels_t*levels_f, 1]))
        energies_by_label_prova[i_a].append(label_to_states([i_a, i_b], [levels_t, levels_f], energies_sys, label_converter))